# Creating the Gymnasium Environment

In [1]:
import gym
import random
import ale_py
import pprint

## Available Environment

In [15]:
all_envs = gym.envs.registry.keys()
[key for key in all_envs]

['ALE/Adventure-v5',
 'ALE/Adventure-ram-v5',
 'ALE/AirRaid-v5',
 'ALE/AirRaid-ram-v5',
 'ALE/Alien-v5',
 'ALE/Alien-ram-v5',
 'ALE/Amidar-v5',
 'ALE/Amidar-ram-v5',
 'ALE/Assault-v5',
 'ALE/Assault-ram-v5',
 'ALE/Asterix-v5',
 'ALE/Asterix-ram-v5',
 'ALE/Asteroids-v5',
 'ALE/Asteroids-ram-v5',
 'ALE/Atlantis-v5',
 'ALE/Atlantis-ram-v5',
 'ALE/Atlantis2-v5',
 'ALE/Atlantis2-ram-v5',
 'ALE/Backgammon-v5',
 'ALE/Backgammon-ram-v5',
 'ALE/BankHeist-v5',
 'ALE/BankHeist-ram-v5',
 'ALE/BasicMath-v5',
 'ALE/BasicMath-ram-v5',
 'ALE/BattleZone-v5',
 'ALE/BattleZone-ram-v5',
 'ALE/BeamRider-v5',
 'ALE/BeamRider-ram-v5',
 'ALE/Berzerk-v5',
 'ALE/Berzerk-ram-v5',
 'ALE/Blackjack-v5',
 'ALE/Blackjack-ram-v5',
 'ALE/Bowling-v5',
 'ALE/Bowling-ram-v5',
 'ALE/Boxing-v5',
 'ALE/Boxing-ram-v5',
 'ALE/Breakout-v5',
 'ALE/Breakout-ram-v5',
 'ALE/Carnival-v5',
 'ALE/Carnival-ram-v5',
 'ALE/Casino-v5',
 'ALE/Casino-ram-v5',
 'ALE/Centipede-v5',
 'ALE/Centipede-ram-v5',
 'ALE/ChopperCommand-v5',
 'ALE/Chop

## Creating Space Invaders Environment

In [3]:
env = gym.make('SpaceInvaders-v0')

In [4]:
height, width, channels = env.observation_space.shape

In [5]:
height, width, channels

(210, 160, 3)

In [6]:
actions = env.action_space.n

In [7]:
actions

6

In [8]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

## 5 Episodes for Trial

In [9]:
episodes = 5

In [11]:
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:105.0
Episode:2 Score:170.0
Episode:3 Score:170.0
Episode:4 Score:210.0
Episode:5 Score:105.0


## Create a Deep Learning Model with Keras 

In [12]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

2025-01-05 15:40:24.022285: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [20]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [21]:
model = build_model(height, width, channels, actions)

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_6 (Dense)              (None, 6)                

## Build Agents with Keras Reinforcement Learning

In [23]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [ ]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [24]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [25]:
dqn = build_agent(model, actions)


In [28]:
dqn.compile(Adam(lr=1e-4))

In [18]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...
 1197/10000: episode: 1, duration: 16.708s, episode steps: 1197, steps per second:  72, episode reward: 275.000, mean reward:  0.230 [ 0.000, 30.000], mean action: 2.595 [0.000, 5.000],  loss: 27.396054, mean_q: 11.414030, mean_eps: 0.901135
 1890/10000: episode: 2, duration: 41.929s, episode steps: 693, steps per second:  17, episode reward: 135.000, mean reward:  0.195 [ 0.000, 30.000], mean action: 2.505 [0.000, 5.000],  loss: 1.129293, mean_q: 9.534999, mean_eps: 0.861130
 2441/10000: episode: 3, duration: 34.082s, episode steps: 551, steps per second:  16, episode reward: 140.000, mean reward:  0.254 [ 0.000, 30.000], mean action: 2.566 [0.000, 5.000],  loss: 1.161340, mean_q: 10.541823, mean_eps: 0.805150
 3449/10000: episode: 4, duration: 61.689s, episode steps: 1008, steps per second:  16, episode reward: 320.000, mean reward:  0.317 [ 0.000, 30.000], mean action: 2.569 [0.000, 5.000],  loss: 1.287153, mean_q: 11.357870, mean_eps: 0.734995
 4513/100

In [32]:
dqn.load_weights('SavedWeights/1m/dqn_weights.h5f')

In [15]:
# import sys
# sys.setrecursionlimit(100000)

In [31]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print("The Average reward is: ", np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 200.000, steps: 668
Episode 2: reward: 345.000, steps: 918
Episode 3: reward: 165.000, steps: 670
Episode 4: reward: 230.000, steps: 868
Episode 5: reward: 300.000, steps: 1220
Episode 6: reward: 435.000, steps: 880
Episode 7: reward: 195.000, steps: 829
Episode 8: reward: 475.000, steps: 860
Episode 9: reward: 330.000, steps: 1420
Episode 10: reward: 225.000, steps: 924
The Average reward is:  290.0
